Reference to TPSMAY22 EDA which makes sense ⭐️⭐️⭐️⭐️⭐️ https://www.kaggle.com/code/ambrosm/tpsmay22-eda-which-makes-sense/notebook
<br>
The ideas about nuunique_char and interactions of three top features comes from that.
<br>
In addition, that is inspired by Analysing Interactions with SHAP https://www.kaggle.com/code/wti200/analysing-interactions-with-shap 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
from scipy import stats

from pandas.plotting import scatter_matrix
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler,LabelEncoder
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.cluster import KMeans
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import Pipeline
import os, glob, math, cv2, gc, logging, warnings, random

# from umap import UMAP

from lightgbm import LGBMClassifier as L
import shap
warnings.filterwarnings("ignore")
print("setup completed")

In [ ]:
# train = pd.read_csv('train.csv')
# test = pd.read_csv('test.csv')
# sub = pd.read_csv('sample_submission.csv')

train = pd.read_csv('../input/tabular-playground-series-may-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2022/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-may-2022/sample_submission.csv')

In [ ]:
train

In [ ]:
sub

In [ ]:
train.info()

In [ ]:
train["target"].value_counts()

## EDA

In [ ]:
feature_cols = [col for col in train.columns if "f_" in col] #f_xxのコラム名抜き出し
dtype_cols = [train[i].dtype for i in feature_cols]
dtypes = pd.DataFrame({"features":feature_cols, "dtype":dtype_cols})
float_cols = dtypes.loc[dtypes["dtype"] == "float64", "features"].values.tolist()
int_cols = dtypes.loc[dtypes["dtype"] == "int64", "features"].values.tolist()

In [ ]:
plt.subplots(figsize=(25,20))
sns.heatmap(train.corr(),annot=True, cmap="RdYlGn", fmt = '0.2f', vmin=-1, vmax=1, cbar=False);

In [ ]:
plt.subplots(figsize=(25,20))
for i, column in enumerate(float_cols):
    plt.subplot(4,4,i+1)
    plt.hist(train[column], bins = 100)
    plt.title(column)

In [ ]:
plt.subplots(figsize=(25,20))
for i, column in enumerate(int_cols):
    plt.subplot(4,4,i+1)
    plt.hist(train[column], bins = 100)
    plt.title(column)

Observations:

- All integers are greater than 0.  They seem to be categorical.
- Attention: f_28 (float) f_29, f_30(int)

## f_27
string object -> label encoding: category

In [ ]:
from sklearn.preprocessing import LabelEncoder as LE

In [ ]:
import string
train.f_27.str.len().min(), train.f_27.str.len().max(), test.f_27.str.len().min(), test.f_27.str.len().max()

In [ ]:
train['f_27'].str[0]

In [ ]:
encoder = LE()
def encode_features(df, cols):
    for col in cols:
        df[col + '_enc'] = encoder.fit_transform(df[col])
    return df

# train = encode_features(train)
# test = encode_features(test)

In [ ]:
# 文字列の1番目～10番目の文字について独立してコラム化＆ラベル化
def str_place_f27(df):
    cols=[]
    for i in range(10):
        df['f_27_'+str(i+1)] = df['f_27'].str[i]
        cols.append('f_27_' + str(i+1))
        df=encode_features(df,cols)        
    return df

train=str_place_f27(train)
test=str_place_f27(test)

In [ ]:
def count_nunique_char(df):
    df['nunique_char']=[len(set(i)) for i in df.f_27]
    return df

train=count_nunique_char(train)
test=count_nunique_char(test)

In [ ]:
print(pd.DataFrame({'n_nunique_char': train.groupby(train.nunique_char).size(), 
       'ratio of target==1': train.groupby(train.nunique_char).target.mean().round(2)}))

## Top three feature interactions

According to TPSMAY22 EDA which makes sense (https://www.kaggle.com/code/ambrosm/tpsmay22-eda-which-makes-sense/notebook)
, which is inspiered by Interaction vs Correlation (https://www.kaggle.com/competitions/tabular-playground-series-may-2022/discussion/323766)

In [ ]:
for df in [train, test]:
    df['i_02_21'] = (df.f_21 + df.f_02 > 5.2).astype(int) - (df.f_21 + df.f_02 < -5.3).astype(int)
    df['i_05_22'] = (df.f_22 + df.f_05 > 5.1).astype(int) - (df.f_22 + df.f_05 < -5.4).astype(int)
    i_00_01_26 = df.f_00 + df.f_01 + df.f_26
    df['i_00_01_26'] = (i_00_01_26 > 5.0).astype(int) - (i_00_01_26 < -5.0).astype(int)


In [ ]:
train.head()

In [ ]:
train.nunique()

In [ ]:
# import pandas_profiling as pdp
# pdp.ProfileReport(train)

## Model

In [ ]:
categorical_features=['f_29', 'f_30', 'i_02_21', 'i_05_22', 'i_00_01_26']
drop=['id', 'target', 'f_27']

for i in range(10):
    categorical_features.append('f_27_' + str(i+1) + '_enc')
    drop.append('f_27_' + str(i+1))

train[categorical_features]=train[categorical_features].astype('category')
test[categorical_features]=test[categorical_features].astype('category')

features = [feat for feat in train.columns if feat not in drop]
target_feature = 'target'

In [ ]:
# from sklearn.model_selection import train_test_split
# test_size_pct = 0.25
# X_train, X_test, y_train, y_test = train_test_split(train[features], train[target_feature], test_size = test_size_pct, random_state = 42)

X_train=train[features]
y_train=train[target_feature]

In [ ]:
# optuna tuning
# from optuna.integration import lightgbm as lgb
# from sklearn.model_selection import train_test_split

# test_size_pct = 0.25
# X_train_op, X_test_op, y_train_op, y_test_op = train_test_split(train[features], train[target_feature], test_size = test_size_pct, random_state = 1)

# dtrain = lgb.Dataset(X_train_op, label=y_train_op)
# eval_data = lgb.Dataset(X_test_op, label=y_test_op)

# param = {'objective': 'binary',
#         'metric': 'binary_logloss',
#         'verbosity': -1,
#         'boosting_type': 'gbdt'
#         }
# best = lgb.train(param, 
#                  dtrain,
#                  valid_sets=eval_data,
#                  early_stopping_rounds=100,
#                  verbose_eval=-1)

In [ ]:
# display(best.params)
# display(best.best_iteration)
# display(best.best_score)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
import lightgbm as lgb
# from optuna.integration import lightgbm as lgb
n_splits=3

kf = StratifiedShuffleSplit(n_splits=n_splits, random_state=0)

# スコアとモデルを格納するリスト
score_lists = []
models = []

for fold_, (train_index, valid_index) in enumerate(kf.split(X_train, y_train)):    
    print(f'fold{fold_ + 1} start')
    train_x = X_train.iloc[train_index]
    valid_x = X_train.iloc[valid_index]
    train_y = y_train[train_index]
    valid_y = y_train[valid_index]

    lgb_train = lgb.Dataset(train_x, train_y,
                            # categorical_feature=categorical_features,
                            free_raw_data=False)
    # 検証用
    lgb_eval = lgb.Dataset(train_x, train_y, reference=lgb_train,
                           # categorical_feature=categorical_features,
                           free_raw_data=False)
    # パラメータを設定
    params = {'task': 'train',                # 学習、トレーニング ⇔　予測predict
              'boosting_type': 'gbdt',        # 勾配ブースティング
              'objective': 'binary',      # 目的関数：多値分類、マルチクラス分類
              'metric': 'binary_logloss',      # 分類モデルの性能を測る指標
              'learning_rate': 0.05,          # 学習率（初期値0.1）
#               'num_leaves': 63,               # 決定木の複雑度を調整（初期値31）
#               'min_data_in_leaf': 15,         # データの最小数（初期値20）
              'num_iterations': 10000,        #100
              
              #provided by optuna
              'lambda_l1': 0.8026786027000106,
              'lambda_l2': 0.5584856380354238,
              'num_leaves': 250,
              'feature_fraction': 0.9840000000000001,
              'bagging_fraction': 0.752143228252002,
              'bagging_freq': 3,
              'min_child_samples': 20,
             }
    
    # 学習
    evaluation_results = {}                                     # 学習の経過を保存する箱
    gbm = lgb.train(params,                                   # 上記で設定したパラメータ
                      lgb_train,                                # 使用するデータセット
                      num_boost_round=10000,                     # 学習の回数
                      valid_names=['train', 'valid'],           # 学習経過で表示する名称
                      # valid_sets=[lgb_train, lgb_eval],    # モデル検証のデータセット
                      valid_sets=[lgb_eval],  
                      evals_result=evaluation_results,          # 学習の経過を保存
                      # categorical_feature=categorical_features, # カテゴリー変数を設定
                      early_stopping_rounds=75,                 # アーリーストッピング# 学習
                      verbose_eval=-1)  
    oof = (gbm.predict(valid_x) > 0.5).astype(int)
    score_lists.append(round(accuracy_score(valid_y, oof)*100,2))
    models.append(gbm)  # 学習が終わったモデルをリストに入れておく
    print(f'fold{fold_ + 1} end\n' )
print(score_lists, '平均score', round(np.mean(score_lists), 2))

In [ ]:
import matplotlib.pyplot as plt

#plt.plot(evals_result['training']['binary_logloss'], label='train_loss')
#plt.plot(evals_result['valid_1']['binary_logloss'], label='valid_loss')
# plt.plot(evaluation_results['train']['binary_log'], label='binary_logloss')
plt.plot(evaluation_results['train']['binary_logloss'], label='binary_logloss')
plt.legend()

In [ ]:
test_pred = np.zeros((len(test), n_splits))

for fold_, gbm in enumerate(models):
    test_pred[:, fold_] = gbm.predict(test[features]) # testを予測

In [ ]:
pred = (np.mean(test_pred, axis=1) > 0.5).astype(int)
sub['target'] = pred
sub.to_csv('submission.csv', index = False)
sub.head()